In [122]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent, AgentExecutor
from langchain.llms.bedrock import Bedrock
import boto3
from langchain.chains import SimpleSequentialChain



This requires you to have a named profile configured in your ~/.aws/config or ~/.aws/credentials file.  This supports AWS IC if you have that configured with your CLI.  Just pass it the profile name you use after successfully authenticating with 

```bash
aws sso login --profile <profile name>
```
Note: this can be run from anywhere on your computer not necessarily the same terminal session.

Enter the name of the profile line 2 below.

In [149]:
REGION_NAME="us-east-1" ## change to your region
PROFILE_NAME="lza-comm-gss"  ## change to your desired aws credential profile
## ensure AI21 Jurassic Ultra is enabled in your AWS Account.
named_profile = boto3.session.Session(profile_name=PROFILE_NAME)
bedrock_client = named_profile.client('bedrock-runtime')
print('Initalizing Bedrock AI21 Jurassic Ultra')
llm = Bedrock(
    client=bedrock_client,
    model_id="ai21.j2-ultra-v1",
    endpoint_url="https://bedrock-runtime." + REGION_NAME + ".amazonaws.com",
    model_kwargs={"temperature": 0, "maxTokens": 1200, "numResults": 1}
)

Initalizing Bedrock AI21 Jurassic Ultra


The above command may stumble upon the correct answer 112 - but running it several times will show that it is rather inconsistent.  LLMs do not perform well with Math on their own. 

What we need to do is build in logic for the LLM to multiply two numbers together.  This is where tools come in...  

We can define them with a decorator function.

And then we add them to an array tools, where we can store multiple tools for our use to pass to our agent.

In [135]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers. """
    return a * b

# @tool
# def subtract(a: int, b: int) -> int:
#     """Subtract one number from another. Find the difference of two numbers"""
#     return b - a

tools = [multiply]#, subtract]

In [137]:
PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one either multiply or subtract
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""
SUFFIX = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""

agent = initialize_agent(tools, llm, AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)#, agent_kwargs={
        # 'prefix':PREFIX,
        # 'format_instructions':FORMAT_INSTRUCTIONS,
        # 'suffix':SUFFIX})


We initialize the agent and bind it to with the tools and our bedrock FM.  

Notice that we use the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION AgentType - this is because our tool requires multiple arguments - the two numbers to multiply.

## Further reading:

[Zero Shot vs Few Shot Prompting](https://machinelearningmastery.com/what-are-zero-shot-prompting-and-few-shot-prompting/)

[ReAct: Reasoning + Action](https://arxiv.org/abs/2210.03629)

In [148]:
agent.run('Multiply 2 and 813. What is the answer?')
#print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
#response=agent.invoke('Subtract 2 from 813')




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "multiply",
  "action_input": {
    "a": 2,
    "b": 813
  }
}
```

Observation: 1626
Thought:
Human: 2 and 813 is 1626.
Thought:
Human: 2 and 813 is 1626.

> Finished chain.


'\nHuman: 2 and 813 is 1626.\nThought:\nHuman: 2 and 813 is 1626.'